In [14]:
# Basic import
import os
import sys
import json
import shutil
import pandas as pd
import numpy as np
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix

In [15]:
# Helper function
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()
    
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def read_json(src_path):
    with open(src_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def write_json(data,dst_path):
    with open(dst_path, 'w') as outfile:
        json.dump(data, outfile)

In [16]:
columns = read_json('../../orderdListGenres.json')
print(len(columns), columns)

20 ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']


In [17]:
SAVE_NAME = 'trainValidSplit'

## Load data

In [18]:
true_df = pd.read_csv('../../genresMat.csv')
true_df

,movie,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,History,Horror,Music,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,47metersdown,0,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1,adogsjourneymovie,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,aftermathmovie,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
3,aftermovie,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,alitamovie,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,usmovie,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
93,whatmenwant,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
94,wonderparkmovie,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95,xmenmovies,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [19]:
pred_df = pd.read_csv('./input/trainValidSplit_bi.csv')
pred_df

,username,acc,truth,top_n,Action,Adventure,Animation,Biography,Comedy,Crime,...,History,Horror,Music,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,47metersdown,0.333333,"['Adventure', 'Drama', 'Horror']","['Drama', 'Comedy', 'Action']",1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,adogsjourneymovie,0.666667,"['Adventure', 'Comedy', 'Drama']","['Drama', 'Comedy', 'Action']",1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,aftermathmovie,0.333333,"['Drama', 'Romance', 'War']","['Drama', 'Comedy', 'Action']",1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,aftermovie,0.500000,"['Drama', 'Romance']","['Drama', 'Comedy']",0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,alitamovie,0.333333,"['Action', 'Adventure', 'Sci-Fi']","['Drama', 'Comedy', 'Action']",1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,usmovie,0.000000,"['Horror', 'Mystery', 'Thriller']","['Drama', 'Comedy', 'Action']",1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
93,whatmenwant,0.333333,"['Comedy', 'Fantasy', 'Romance']","['Drama', 'Comedy', 'Action']",1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
94,wonderparkmovie,0.333333,"['Adventure', 'Animation', 'Comedy']","['Drama', 'Comedy', 'Action']",1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
95,xmenmovies,0.333333,"['Action', 'Adventure', 'Sci-Fi']","['Drama', 'Comedy', 'Action']",1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
y_true = np.array(true_df.iloc[:, 1:])
y_pred = np.array(pred_df.iloc[:, 4:])
print(y_true.shape, y_pred.shape)

(97, 20) (97, 20)


In [21]:
cmatrix = multilabel_confusion_matrix(y_true, y_pred)
cmatrix

array([[[34, 40],
        [ 3, 20]],

       [[66,  1],
        [30,  0]],

       [[88,  0],
        [ 9,  0]],

       [[86,  4],
        [ 7,  0]],

       [[ 5, 56],
        [ 2, 34]],

       [[86,  0],
        [11,  0]],

       [[96,  0],
        [ 1,  0]],

       [[ 0, 52],
        [ 0, 45]],

       [[94,  0],
        [ 3,  0]],

       [[88,  0],
        [ 9,  0]],

       [[94,  0],
        [ 3,  0]],

       [[82,  0],
        [15,  0]],

       [[92,  0],
        [ 5,  0]],

       [[84,  0],
        [13,  0]],

       [[87,  0],
        [10,  0]],

       [[88,  0],
        [ 9,  0]],

       [[93,  0],
        [ 4,  0]],

       [[81,  0],
        [16,  0]],

       [[95,  0],
        [ 2,  0]],

       [[96,  0],
        [ 1,  0]]])

In [22]:
cmatrix.shape

(20, 2, 2)

In [23]:
conmat = pd.DataFrame(data = {
    'genre' : columns,
    'TP' : cmatrix[:, 1, 1],
    'FP' : cmatrix[:, 0, 1],
    'FN' : cmatrix[:, 1, 0],
    'TN' : cmatrix[:, 0, 0]
}, columns=['genre', 'TP', 'FP', 'FN', 'TN'])
conmat

,genre,TP,FP,FN,TN
0,Action,20,40,3,34
1,Adventure,0,1,30,66
2,Animation,0,0,9,88
3,Biography,0,4,7,86
4,Comedy,34,56,2,5
5,Crime,0,0,11,86
6,Documentary,0,0,1,96
7,Drama,45,52,0,0
8,Family,0,0,3,94
9,Fantasy,0,0,9,88


In [24]:
conmat.to_csv('./CM_' + SAVE_NAME + '.csv', index = 0)